# DicoCo

*anis SAHLI*

## Récupération de la Data

In [17]:
# Récupération de la Data
import pandas as pd
dfBuchananRich = pd.read_csv('./../data/dataframe/BuchananRich.csv')

In [18]:
dfBuchananRich=dfBuchananRich[["motVedette","nbSyll","pron","pronSampa","ipa"]]
dfBuchananRich.head()

,motVedette,nbSyll,pron,pronSampa,ipa
0,abacus,3,aʹbăcus,{ b @ k @ s,æbəkəs
1,abaft,2,ăbăʹft,mot inexistant,mot inexistant
2,abaisance,3,ăbaiʹſănce,mot inexistant,mot inexistant
3,abandon,3,ābăʹndon,@ b { n d @ n,əbændən
4,abandoned,4,ābăʹndŏned,@ b { n d @ n d,əbændənd


In [19]:
# attention le mot null lors de l'import est considéré comme une missing value
dfBuchananRich[dfBuchananRich["motVedette"].isnull()]

,motVedette,nbSyll,pron,pronSampa,ipa
9193,NaN,1,nŭll,mot inexistant,mot inexistant


In [20]:
dfBuchananRich["motVedette"].fillna("null",inplace=True)
dfBuchananRich[dfBuchananRich["motVedette"].isnull()]

,motVedette,nbSyll,pron,pronSampa,ipa


## Scrapping

In [21]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import re
import time

session = HTMLSession()

In [22]:
# Scrapper une fois toute la table
# Test sur les 100 premiers
path = "https://dictionary.cambridge.org/fr/dictionnaire/anglais/"

compteur = 0

for word in dfBuchananRich["motVedette"].iloc[0:10]:
    response = session.get(path+word)
    status = response.status_code
    if status != 200: 
        print("Attention, au mot {} : status code = {}".format(word,status))
    
    time.sleep(0.1)
    
if compteur == 0 :
    print("Aucun problème à priori sur l'ensemble des mots!")

Aucun problème à priori sur l'ensemble des mots!


## Fonction de scraping 

In [23]:
# Scrapping version 2
path = "https://dictionary.cambridge.org/fr/dictionnaire/anglais/"

def cambridge_scrap2(word,session):
    list_of_words = []
    list_of_type = []
    list_uk_pron = []
    list_phonetic_uk = []
    list_us_pron = []
    list_phonetic_us = []

    response = session.get(path+word)
    status = response.status_code
    if status == 200: 
        html_doc = response.content
        soup = BeautifulSoup(html_doc,'html.parser')

        finded = soup.findAll("div", {"class" : "pos-header dpos-h"})
        path_audio = 'https://dictionary.cambridge.org'

        for audio in finded :
            is_word_to_catch = False
            try :
                mot = audio.find("div",{"class":"di-title"}).text
                #print(mot)

                dtype = audio.find("div",{"class":"posgram dpos-g hdib lmr-5"}).text
                #print('type : {}'.format(dtype))
                
                list_of_words.append(mot)
                list_of_type.append(dtype)
                
                is_word_to_catch = True
            except :
                pass
            
            #print(is_word_to_catch)

            if(is_word_to_catch==True):
                #print(mot)
                # UK
                try:
                    langue_uk = audio.find("span",{"class":"uk dpron-i"})
                    if len(langue_uk)==0:
                        pass
                    else:
                        phonetic_uk = audio.find("span",{"class":"pron dpron"}).text
                        list_uk_pron.append(path_audio+langue_uk.find("source",{"type":"audio/mpeg"})['src'])
                        list_phonetic_uk.append(phonetic_uk)
                except:
                    list_uk_pron.append(None)
                    list_phonetic_uk.append(None)
                    #print("pas de uk")

                # US
                try:
                    langue_us = audio.find("span",{"class":"us dpron-i"})
                    if len(langue_us)==0:
                        pass
                    else:
                        phonetic_us = audio.find("span",{"class":"pron dpron"}).text
                        list_us_pron.append(path_audio+langue_us.find("source",{"type":"audio/mpeg"})['src'])
                        list_phonetic_us.append(phonetic_us)
                except:
                    list_us_pron.append(None)
                    list_phonetic_us.append(None)
                    #print("pas de uk")

    dico_result = {
        'word' : list_of_words,
        'POS' : list_of_type,
        'UKpron' : list_uk_pron,
        'UKphonetic' : list_phonetic_uk,
        'USpron' : list_us_pron,
        'USphonetic' : list_phonetic_us
    }

    return(dico_result)

## Sortie

In [24]:
# import necessaire à la création des widgets
from IPython.display import display, clear_output, HTML, Markdown 
import IPython.display as ipd
import ipywidgets as widgets

In [25]:
# definition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="",
       description='hwd',
       layout={'width': '180px'} ) #Taille de la zone de saisie

textPronSampa = widgets.Text(
       value="",
       description='pronSampa',
       layout={'width': '180px'} ) #Taille de la zone de saisie

textIpa = widgets.Text(
       value="",
       description='ipa',
       layout={'width': '180px'} ) #Taille de la zone de saisie

In [26]:
# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer',width=100)
occ_max = widgets.IntSlider(max=20, value=1,description='nb occ.')

# définition du widget permettant l'affichage d'une sortie
out1 = widgets.Output()
#out2 = widgets.Output()

# spécifier les actions à réaliser lors de l'appui sur le bouton
def on_button_clicked(_):
    
    # récupérer le texte saisi par l'utilisateur
    targetWord = textTarget.value # est une regex
    targetWordSampa = textPronSampa.value # est une regex
    targetWordIpa = textIpa.value # est une regex

#    with out1:
#        clear_output()
#        
#        targetWord = textTarget.value
#        urlLinguee = "https://dictionary.cambridge.org/fr/dictionnaire/anglais/"
#        urlTarget = urlLinguee+targetWord+'.html'
#        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'cambrigde : '+targetWord+'</a>'))

    with out1:
        clear_output()
        
        df_filtered = dfBuchananRich.copy()
        
        # Premier filtre : targetWord
        if(len(targetWord)>0):
            # Filtrer le dataframe avec les Inputs
            series = df_filtered["motVedette"]
            series_filtered = series.str.contains(targetWord)
            df_filtered = df_filtered[series_filtered]
            
        # Deuxième filtre : targetWordSampa
        # Premier filtre : targetWord
        if (len(targetWordSampa)>0):
            # Filtrer le dataframe avec les Inputs
            series = df_filtered["pronSampa"]
            series_filtered = series.str.contains(targetWordSampa)
            df_filtered = df_filtered[series_filtered]
            
        # Troisième filtre : targetWordIpa
        if (len(targetWordIpa)>0):
            # Filtrer le dataframe avec les Inputs
            series = df_filtered["ipa"]
            series_filtered = series.str.contains(targetWordIpa)
            df_filtered = df_filtered[series_filtered]
        
        df_filtered = df_filtered.iloc[0:occ_max.value]
        #display(df_filtered)
        display(HTML(df_filtered.to_html(index=False)))
        
        #afficher le résultat
        words = [filtered_word for filtered_word in df_filtered['motVedette']]
        for actual_word in words:
            #caption = widgets.Label(value=actual_word)
            # Mise en forme du text
            
            caption = widgets.Label(value = r'\(\color{red} { Mot :' + actual_word  + '}\)')
            display(caption)
            resultScrap = pd.DataFrame(cambridge_scrap2(actual_word,session))
            #display(resultScrap)
            
            # UKpron
            # UKphonetic
            # USpron
            # USphonetic
            for index, row in resultScrap.iterrows():
                pos = row['POS']
                display(widgets.Label(value="{}. {}".format(index+1,pos)))
                
                audio_widgets = []
                listUrl=[row['UKpron'],row['USpron']]
                
                audio_widgets = list()
                for urlCur in listUrl:
                    if (urlCur is not None):
                        if('uk_pron' in urlCur):
                            #print('uk_pron')
                            curent_phonetic=row['UKphonetic']
                            langue = widgets.Label(value = 'UK (phonetic : {}) : '.format(curent_phonetic))
                            langue.font_size=20
                        else:
                            #print('us_pron')
                            curent_phonetic=row['USphonetic']
                            langue = widgets.Label(value = 'US (phonetic : {}) : '.format(curent_phonetic))
                            langue.font_size=20
                        
                        audio_widgets.append(langue)
                        
                        out = widgets.Output()
                        with out:
                            clear_output()
                            ipd.display(ipd.Audio(url=urlCur))
                        audio_widgets.append(out)
                        
                    box = widgets.HBox(tuple(audio_widgets))
                display(box)
                
button.on_click(on_button_clicked)

# les widgets de filtre input
# Objet de type widget layout horizontal
vbFiltres = widgets.HBox([textTarget,textPronSampa,textIpa,occ_max , button])

# les widgets d'output
# Objet de type widget layout vertical
vbOuts = widgets.VBox([out1])

hbAllWidgets = widgets.VBox([vbFiltres,vbOuts])

hbAllWidgets